In [59]:
import math
from timeit import default_timer as timer

import bayesfunc as bf
import matplotlib
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
from torch.distributions import Categorical, Normal
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader

from data.regression import RegressionDataset
from tqdm.auto import tqdm
import numpy as np

%load_ext autoreload
%autoreload 2

matplotlib.style.use('default')
plt.rcParams.update({'axes.titlesize': 'large', 'axes.labelsize': 'medium'})
colors_hex = {'blue': '#1F77B4', 'orange': '#FF7F0E', 'green': '#2CA02C'}
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
t.cuda.is_available()
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
print(device)

t.manual_seed(0)

cuda:0


How to download, init and use a UCI dataset

In [61]:
from uci import UCI

In [62]:
protein = UCI('protein', 1, 10000)
protein.trainset

Using Ashman's code

In [63]:
from data.preprocess_data import download_datasets, process_dataset, datasets, protein_config
import os

In [64]:
dir_path = "./data/uci"
download_datasets(root_dir='./data/uci', datasets={'protein': datasets['protein']})
process_dataset(os.path.join(dir_path, "protein"), protein_config)

protein dataset already exists!
Input  shape: (45730, 9)
Output shape: (45730, 1)


In [65]:
data_dir = lambda x: os.path.join(dir_path, "protein", x)

X, y = np.load(data_dir("x.npy")), np.load(data_dir("y.npy"))

In [66]:
from config import DataLoaderConfig, HyperParamConfig
from dataclasses import asdict
from data import ListDataset

In [67]:
params = HyperParamConfig(
    name = "Regression - UCI",

    # Dimensions
    batch_size=128,
    epochs = 1000,
    input_dim = 9,
    output_dim = 1,
    hidden_units = 100,

    # Training
    elbo_samples = 5,
    inference_samples = 10,
    regression_likelihood_noise = 0.1,

    # Optimiser
    lr = 1e-3,
    step_size = 100, 
    gamma = 0.10,
    shuffle = True
)

In [68]:
dataset = ListDataset(X, y)
N = len(dataset)
train_test_split = 0.8
fraction = math.ceil(train_test_split*N)
train, test = t.utils.data.random_split(dataset, [fraction, N-fraction])

train_loader, test_loader = DataLoader(train, **(params.dataloader)), DataLoader(test, **(params.dataloader))


### BNN

In [69]:
# Set inducing set U_0
inducing_batch = 100
inducing_data, inducing_targets = next(iter(train_loader))
 
# for classification
# inducing_targets = (t.arange(num_classes) == inducing_targets[:, None]).float()

# Initialize to first <inducing_batch> training samples
if inducing_batch < inducing_data.shape[0]:
    inducing_data = inducing_data[:inducing_batch]
    inducing_targets = inducing_targets[:inducing_batch]

# Fill up inducing set with random initialized points on [0,1] if batch smaller than num inducing pts
if inducing_batch > inducing_data.shape[0]:
    inducing_data = t.cat([inducing_data, t.randn(inducing_batch-inducing_data.shape[0], *inducing_data.shape[1:], dtype=inducing_data.dtype)], 0)
    
    inducing_targets = t.cat(
        [inducing_targets, t.randn(inducing_batch - inducing_targets.shape[0], *inducing_targets.shape[1:], dtype=inducing_targets.dtype)], 0)
in_features = inducing_data.shape[-1]

In [70]:
from bayesfunc.priors import InsanePrior, NealPrior

optional_layer_params = {
    # 'bias': True,
    'prior': NealPrior, #InsanePrior,
    #'inducing_targets': initial value of inducing targets (default None)
    'log_prec_init': -4, # initial precision parameter values; default -4, assumes little data available
    'log_prec_lr': 1, # LR multiplier for precision params
    # 'inducing_targets': inducing_targets
}

hidden_units=50
bnn_net = nn.Sequential(
    bf.GILinear(in_features=params.input_dim, out_features=hidden_units, inducing_batch=inducing_batch, bias=True, **optional_layer_params),
    nn.ReLU(),
    bf.GILinear(in_features=hidden_units, out_features=hidden_units, inducing_batch=inducing_batch, bias=True, **optional_layer_params),
    nn.ReLU(),
    bf.GILinear(in_features=hidden_units, out_features=params.output_dim, inducing_batch=inducing_batch, bias=True, full_prec=True, **optional_layer_params)
)

# Wrap model in inducing wrapper
dtype = t.float64
# bnn_net = bf.InducingWrapper(bnn_net, inducing_shape=(inducing_batch, in_features), inducing_batch=inducing_batch)
bnn_net = bf.InducingWrapper(bnn_net, inducing_data=inducing_data, inducing_batch=inducing_batch)
# Send to GPU
bnn_net = bnn_net.to(device=device, dtype=dtype)

In [71]:
def train(net, train_loader, epoch, scale, factor, num_samples=10, plot=False):
    iters = 0
    total_elbo = 0.
    total_ll = 0.
    total_KL = 0.
    total_error = 0.

    # Tempered beta to have loss not be KL divergence (to optimize eval performance)
    temper = False
    L = 1. # temperature scaling
    tempered_beta = 0.1*math.floor((epoch-1)/10.)/L if (temper and epoch < 100) else 1/L
    beta = 1/L

    
    for data, target in train_loader:
        batch_size = data.shape[0]
        
        opt.zero_grad() # zero out the gradient (Torch accumulates across epochs otherwise)
        
        data, target = data.to(device), target.to(device)
        data = data.expand(num_samples, *data.shape) # make it S x N x D
        
        outputs, logPQw, _ = bf.propagate(net, data)
        
        # Classification: compute log likelihood estimate via outputs 
        # dist| = Categorical(logits=outputs)
        # ll = dist.log_prob(target).mean() 
        
        def log_s2():
            return scale*factor

        # Regression:
        ll = Normal(outputs.to(device), t.exp(0.5*log_s2())).log_prob(target) # [10, batch size, output] => need to take mean across inference samples (first dim)
        ll = ll.mean(0)
        ll = ll.sum() # across batch size
        
        # Compute objectives
        nloss = ll.mean() + tempered_beta * logPQw.mean()/ batch_size  # tempered ELBO
        elbo = ll + beta * (logPQw / batch_size)

        (-elbo.mean()).backward()  
        opt.step()

        # For classification only
        # output = outputs.log_softmax(-1).logsumexp(0) - math.log(outputs.shape[0])
        # pred = output.argmax(dim=-1, keepdim=True)
        # correct = pred.eq(target.view_as(pred)).float().mean()
        
        error = (outputs.mean() - target)
        
        iters         += 1
        total_elbo    += elbo.mean().item()
        total_ll      += ll.mean().item()
        total_KL      -= (beta*logPQw.mean()/batch_size).item()
        total_error   += error.sum()

    return (total_elbo/iters, total_ll/iters, total_KL/iters, total_error/iters)

In [72]:
def test(net, loader, scale, num_samples=10):
    iters = 0
    total_elbo = 0.
    total_ll = 0.
    total_KL = 0.
    total_error = 0.
    
    for data, target in loader:
        batch_size = data.shape[0]
                
        data, target = data.to(device), target.to(device)
        data = data.expand(num_samples, *data.shape) # make it S x N x D
        
        outputs, logPQw, _ = bf.propagate(net, data)
        
        # Classification: compute log likelihood estimate via outputs 
        # dist| = Categorical(logits=outputs)
        # ll = dist.log_prob(target).mean() 
        
        def log_s2():
            return scale*factor

        # Regression:
        ll = Normal(outputs.to(device), t.exp(0.5*log_s2())).log_prob(target) # [10, batch size, output] => need to take mean across inference samples (first dim)
        ll = ll.mean(0)
        ll = ll.sum() # across batch size
        
        # Compute objectives
        elbo = ll + (logPQw / batch_size)
        error = (outputs.mean() - target) # regression

        # For classification only
        # output = outputs.log_softmax(-1).logsumexp(0) - math.log(outputs.shape[0])
        # pred = output.argmax(dim=-1, keepdim=True)
        # correct = pred.eq(target.view_as(pred)).float().mean()
        
        iters         += 1
        total_ll      += ll.mean().item()
        total_error   += error.sum()

    return (total_ll/iters, total_error/iters)

In [76]:
net = bnn_net
loader = train_loader

lr = 0.05
factor=10
log_s2_scaled = t.tensor(-3./factor, requires_grad=True, device=device)
opt = t.optim.Adam([*net.parameters(), log_s2_scaled], lr=lr)
scheduler = ExponentialLR(opt, gamma=0.95)
epochs = 10

epoch = []
train_elbo = []
train_ll = []
train_KL = []
test_ll = []
train_err = []
test_err = []

pbar = tqdm(range(epochs), position=0, leave=True)
start_time = timer()
prev_time = timer()
for _epoch in pbar:
    
    _elbo, _train_ll, _train_KL, _train_err = train(net, loader, _epoch, log_s2_scaled, factor)

    if _epoch % 100 == 0: scheduler.step()

    epoch.append(_epoch)
    train_elbo.append(_elbo)
    train_ll.append(_train_ll)
    train_KL.append(_train_KL)
    train_err.append(_train_err)
    

    with t.no_grad():
        _test_ll, _test_err = test(net, test_loader, log_s2_scaled)
    test_ll.append(_test_ll)
    test_err.append(_test_err)

    if _epoch%10 == 0:
        report_time = timer() - prev_time
        prev_time += report_time
        total_time = timer() - start_time

        report = f"epoch: {_epoch:03d}, time: {report_time: 3.2f} (total {total_time:3.2f}), elbo: {_elbo:.3f}, KL: {_train_KL:.3f}, train err: {_train_err:.3f}, test err: {_test_err:.3f}"
        pbar.set_description(report)
    
print(train_elbo[-1])

epoch: 000, time:  4.16 (total 4.16), elbo: -51.023, KL: 5.206, train err: 0.201, test err: 1.709: 100%|██████████| 10/10 [00:41<00:00,  4.14s/it]

-52.25177021986619


In [ ]:
def plot(net, test_loader: DataLoader=None, num_inference_samples=10):
    xs = []
    targets = []
    preds = []
    stds = []

    with t.no_grad(): # no grad computation

        for data, target in test_loader:

            data, target = data.to(device), target.to(device)
            reshaped_data = data.expand(num_inference_samples, *data.shape) # make it S x N x D
        
            outputs, logPQw, sample_dict = bf.propagate(bnn_net, reshaped_data)
            
            pm = outputs.mean(0).flatten().cpu() 
            ps = outputs.std(0).flatten().cpu()

            xs.append(data)
            targets.append(target)
            preds.append(pm)
            stds.append(ps)


        xs = t.cat(xs, dim=1).cpu() # shape: inference_samples x batch_size/N x 1
        xs = xs.flatten()
        preds = t.cat(preds, dim=0).cpu()
        targets = t.cat(targets, dim=1).cpu() # shape: N x 1
        targets = targets.flatten()
        stds = t.cat(stds, dim=0).cpu()

        # Sort in same order
        xs, preds, targets, stds = (np.array(t) for t in zip(*sorted(zip(xs, preds, targets, stds))))

        plt.fill_between(xs, preds-2*stds, preds+2*stds, alpha=0.5)
        plt.plot(xs, preds, label='Prediction mean')
        plt.scatter(xs, targets, label='True values')
        plt.show()

        return xs, preds, targets, stds